# Predicting Fuel Consumption with Pytorch

In [1]:
# imports
import torch, torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.nn import Module
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import os
os.chdir('c:/users/nicolas/documents/data/thecarconnection')

In [2]:
# loading the data
df = pd.read_csv('cleaned_encoded_data.csv')

In [3]:
# dropping columns, assigning input/output
x = df.drop(['Make', 'Model', 'Style Name', 'EPA Fuel Economy Est - Hwy (MPG)',
             'EPA Fuel Economy Est - City (MPG)'], axis=1).values
y = df['EPA Fuel Economy Est - Hwy (MPG)'].values

In [4]:
# casting the input/output
x = x.astype(np.float32)
y = y.astype(np.float32)

In [5]:
# rescale the predictors
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

In [6]:
# send to cuda if available
if torch.cuda.is_available():
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)

In [7]:
# create sampler
class CarDataset():
    
    def __init__(self):
        self.len = x.shape[0]
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return x[index], y[index].unsqueeze(0)
    
    def __len__(self):
        return self.len

In [8]:
# create an instance of sampler
data = CarDataset()

In [9]:
# create iterator
train_loader = DataLoader(dataset=data, batch_size=8, shuffle=True)

In [10]:
# create model class
class Model(Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(270, 156)
        self.l2 = torch.nn.Linear(156, 312)
        self.l3 = torch.nn.Linear(312, 1024)
        self.l4 = torch.nn.Linear(1024, 1)
        
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        out1 = self.relu(self.l1(x))
        out2 = self.relu(self.l2(out1))
        out3 = self.relu(self.l3(out2))
        ypred = self.l4(out3)
        return ypred

In [11]:
# instantiate the model
model = Model()

In [12]:
# loss function and optimizer
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=.0001)

In [13]:
# train the model 
for epoch in range(1, 10+1):
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        ypred = model(inputs)
        loss = criterion(ypred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch', str(epoch) + ' Loss:', np.round(loss.data.item(), 3))
    if loss.data.item() < .5:
        break

Epoch 1 Loss: 0.996
Epoch 2 Loss: 1.236
Epoch 3 Loss: 1.253
Epoch 4 Loss: 1.03
Epoch 5 Loss: 0.613
Epoch 6 Loss: 1.046
Epoch 7 Loss: 1.132
Epoch 8 Loss: 0.702
Epoch 9 Loss: 1.585
Epoch 10 Loss: 0.851
